In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
from datetime import datetime
import plotly.express as px
from skimage import io
import math
import time

In [ ]:
Sunday = pd.DataFrame(pd.read_csv('park-movement-Sun.csv'))
Sunday = Sunday[:-1]
Sunday = Sunday.dropna()
Sunday["id"] = Sunday["id"].astype(str)
Sunday = Sunday.replace('check-in', 1).replace('movement', 0)
Sunday["Timestamp"] = Sunday["Timestamp"].map(lambda x: int(time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))))
Sunday["visit_count"] = Sunday.groupby(["id"])["id"].transform("count")
T = pd.DataFrame(Sunday.groupby('id')['Timestamp'].agg(np.ptp)).reset_index()
T["visit_duration"] = T['Timestamp'].map(lambda x:x//60)
Sunday_II = Sunday.merge(T[["id", "visit_duration"]])
Sunday_II["Facility_coordinates"]= Sunday_II.apply(lambda row: (int(row.X),int(row.Y)), axis=1)
Sunday_II["X"] = Sunday_II["X"].map(lambda x:int(x))
Sunday_II["Y"] = Sunday_II["Y"].map(lambda x:int(x))
# #Sunday_II.to_csv('park-movement-Sun_New.csv')

In [ ]:
Sunday_II = pd.DataFrame(pd.read_csv('park-movement-Sun_New.csv'))
Sunday_II = Sunday_II.drop(["Unnamed: 0"], axis=1)
Sunday_II["id"] = Sunday_II["id"].astype(str)
Sunday_II["Facility_coordinates"]= Sunday_II.apply(lambda row: (int(row.X),int(row.Y)), axis=1)
Sunday_II["X"] = Sunday_II["X"].map(lambda x:int(x))
Sunday_II["Y"] = Sunday_II["Y"].map(lambda x:int(x))

In [ ]:
Sunday_II

In [ ]:
Sunday_II.isna().sum()

In [ ]:
Sunday_II.info()

In [ ]:
px.density_heatmap(Sunday_II, x='X', y='Y')

In [ ]:
counts, bins = np.histogram(Sunday_II.Timestamp, bins=range(Sunday_II["Timestamp"].min(), Sunday_II["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

### Criterion One：Relative time difference and relative distance (Euclidean distance)

In [ ]:
Group = []
# Families, relatives, couples, or any close small group
Crowd = []
# Groups of people who do not belong to each other in small groups, but have similar behavioral patterns
Single = []
# Tourists who do not belong to the above two groups

for index, ID in enumerate(Sunday_II.id.unique()):
    D = Sunday_II.loc[Sunday_II['id'] == ID] 
    Trajectory_I  = D[['id','Timestamp', 'type', "Facility_coordinates"]].sort_values(by=['Timestamp']).to_numpy() 

    Len1 = len(Trajectory_I)
    
    for index2, ID2 in enumerate(Sunday_II.id.unique()[index:]):
        if index2 > 10: 
            break
        if ID == ID2: 
            continue
            
        D2 = Sunday_II.loc[Sunday_II['id'] == ID2] 
        Trajectory_II = D2[['id','Timestamp', 'type', "Facility_coordinates"]].sort_values(by=['Timestamp']).to_numpy()
        Len2 = len(Trajectory_II)
        
        
        if Len1 == Len2:
            track_1 =  pd.Series(list(Trajectory_I[:,3]))
            track_2 =  pd.Series(list(Trajectory_II[:,3]))
            distance = list( track_1.combine(track_2, (lambda x1, x2: 1 if np.linalg.norm(np.array(x1) - np.array(x2)) <= 2 else 0 )))
            Ddiff = distance.count(1)/len(distance)            
            
            if Ddiff >= 0.7:
        
                Tdiff = np.median(np.abs(Trajectory_I[:,1]-Trajectory_II[:,1]))

                if Tdiff <= 60:
                    Group.append(ID)
                    Group.append(ID2)
                    
                else:
                    Crowd.append(ID)
                    Crowd.append(ID2)

                    
        else:
            track_1 =  pd.Series(list(Trajectory_I[:,3]))
            track_2 =  pd.Series(list(Trajectory_II[:,3]))
            distance = list( track_1.combine(track_2, (lambda x1, x2: 1 if np.linalg.norm(np.array(x1) - np.array(x2)) <= 2 else 0 )))
            Ddiff = distance.count(1)/len(distance)
        
            if Ddiff >= 0.7:
                Crowd.append(ID)
                Crowd.append(ID2)

            else:
                Single.append(ID)
                Single.append(ID2)


In [ ]:
d1 = pd.read_csv('SundayGroupCount.csv')
d2 = pd.read_csv('SundayCrowdCount.csv')
d3 = pd.read_csv('SundaySingleCount.csv')

Group = [  i.split(".", 1)[0] for i in d1.columns ]
Crowd = [  i.split(".", 1)[0] for i in d2.columns ]
Single = [  i.split(".", 1)[0] for i in d3.columns ]

In [ ]:
Sunday_Group = list(set(Group))
Sunday_Crowd = list(set(Crowd)-set(Group))
Sunday_Single=  list(set(list(set(Single)-set(Group)))-set(Crowd)) 

In [ ]:
len(Sunday_II.id.unique()),len(Sunday_Group), len(Sunday_Crowd), len(Sunday_Single)

### Criterion Two：Earliest arrival time grouping

In [ ]:
TimePointMorning = time.mktime(time.strptime("2014-6-08 11:00:00", "%Y-%m-%d %H:%M:%S"))
TimePointAfterNoon = time.mktime(time.strptime("2014-6-08 16:00:00", "%Y-%m-%d %H:%M:%S"))

ArrivalTime = Sunday_II.copy().drop_duplicates(subset='id')
ArrivalTime["ArrivalTime"] = ArrivalTime["Timestamp"].map(lambda x: 1 if x- TimePointMorning >= 0 and x - TimePointAfterNoon <= 0 else ( 2 if x - TimePointAfterNoon >= 0 else 0 ) )
Early = list(ArrivalTime[ArrivalTime["ArrivalTime"]==0].id.unique())
Early = [  i.split(".", 1)[0] for i in Early ]

Noon =  list(ArrivalTime[ArrivalTime["ArrivalTime"]==1].id.unique())
Noon = [  i.split(".", 1)[0] for i in Noon ]

Late = list(ArrivalTime[ArrivalTime["ArrivalTime"]==2].id.unique())
Late = [  i.split(".", 1)[0] for i in Late ]

len(Sunday_II.id.unique()), len(Early), len(Noon), len(Late)

### Criterion Three：duration stay time

In [ ]:
m = np.mean(Sunday_II["visit_duration"].unique())
print(m)

In [ ]:
Sunday_IITime = Sunday_II.copy().drop_duplicates(subset='id')
HighDuration = list(Sunday_IITime[Sunday_IITime["visit_duration"] >= m ].id)
HighDuration = [  i.split(".", 1)[0] for i in HighDuration ]

LowDuration = list(Sunday_IITime[Sunday_IITime["visit_duration"] < m ].id)
LowDuration = [  i.split(".", 1)[0] for i in LowDuration ]

len(Sunday_II.id.unique()), len(HighDuration), len(LowDuration)

In [ ]:
Sunday_II['id'] = Sunday_II['id'].map(lambda x:x[:-2])

#### Group 1

In [ ]:
G1 = list(set(Sunday_Group)&set(Early)&set( HighDuration))
G1P = Sunday_II.loc[Sunday_II['id'].isin(G1)]
len(G1)

In [ ]:
px.density_heatmap(G1P, x='X', y='Y')

In [ ]:
counts, bins = np.histogram(G1P.Timestamp, bins=range(G1P["Timestamp"].min(), G1P["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

#### Group 2

In [ ]:
G2 = list(set(Sunday_Crowd)&set(Early)&set( HighDuration))
G2P = Sunday_II.loc[Sunday_II['id'].isin(G2)]
len(G2)

In [ ]:
px.density_heatmap(G2P, x='X', y='Y')

In [ ]:
counts, bins = np.histogram(G2P.Timestamp, bins=range(G2P["Timestamp"].min(), G2P["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

#### Group 3

In [ ]:
G3 = list(set(Sunday_Single)&set(Early)&set(HighDuration))
G3P = Sunday_II.loc[Sunday_II['id'].isin(G3)]
len(G3)

In [ ]:
px.density_heatmap(G3P, x='X', y='Y')

In [ ]:
counts, bins = np.histogram(G3P.Timestamp, bins=range(G3P["Timestamp"].min(), G3P["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

#### Group 4

In [ ]:
G4 = list(set(Sunday_Group)&set(Noon))
G4P = Sunday_II.loc[Sunday_II['id'].isin(G4)]
len(G4) 

In [ ]:
px.density_heatmap(G4P, x='X', y='Y')

In [ ]:
counts, bins = np.histogram(G4P.Timestamp, bins=range(G4P["Timestamp"].min(), G4P["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

#### Group 5

In [ ]:
G5 = list(set(Sunday_Crowd)&set(Noon))
G5P = Sunday_II.loc[Sunday_II['id'].isin(G5)]
len(G5) 

In [ ]:
px.density_heatmap(G5P, x='X', y='Y')

In [ ]:
counts, bins = np.histogram(G5P.Timestamp, bins=range(G5P["Timestamp"].min(), G5P["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

#### Group 6 

In [ ]:
G6 = list(set(Sunday_Single)&set(LowDuration))
G6P = Sunday_II.loc[Sunday_II['id'].isin(G6)]
len(G6) 

In [ ]:
px.density_heatmap(G6P, x='X', y='Y')

In [ ]:
counts, bins = np.histogram(G6P.Timestamp, bins=range(G6P["Timestamp"].min(), G6P["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

#### Group 7 

In [ ]:
G7 = list(set(Sunday_Group)&set(LowDuration)&set(Early))
G7P = Sunday_II.loc[Sunday_II['id'].isin(G7)]
len(G7)

In [ ]:
px.density_heatmap(G7P, x='X', y='Y')

In [ ]:
counts, bins = np.histogram(G7P.Timestamp, bins=range(G7P["Timestamp"].min(), G7P["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

#### Group 8

In [ ]:
G8 = list(set(Sunday_Group)&set(Late))
G8P = Sunday_II.loc[Sunday_II['id'].isin(G8)]
len(G8)

In [ ]:
px.density_heatmap(G8P, x='X', y='Y')

In [ ]:
counts, bins = np.histogram(G8P.Timestamp, bins=range(G8P["Timestamp"].min(), G8P["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

### Criterion Four, the time interval between the activities of each visitor

In [ ]:
visitors_td = []
max_td = []
med_td = []
for index, ID in enumerate(Sunday_II.id.unique()):
    visitors = Sunday_II.loc[Sunday_II['id'] == ID] 
    l = len(visitors['Timestamp'].values)
    diff  = abs(visitors['Timestamp'].values[1:] - Sunday_II['Timestamp'].values[0:l-1])
    # Calculate the time difference between the tourist's first action and the next second action
    dmax = max(diff)
    dmed = np.median(diff)
    visitors_td.append(ID)
    max_td.append(dmax)
    med_td.append(dmed)

###  Visitors with huge gaps between actions (top 150), sorted from largest to smallest

In [ ]:
inds1 =  np.argsort(max_td)
GapH = np.array(visitors_td)[inds1[-150:]]
GapHV = Sunday_II.loc[Sunday_II['id'].isin(GapH)]
px.density_heatmap(GapHV, x=GapHV['X'], y=GapHV['Y'])

In [ ]:
counts, bins = np.histogram(GapHV.Timestamp, bins=range(GapHV["Timestamp"].min(), GapHV["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

### Fast-moving tourists (low gap between actions)

In [ ]:
inds2 =  np.argsort(med_td) 
Faster = np.array(visitors_td)[inds2[0:150]]
FasterV = Sunday_II.loc[Sunday_II['id'].isin(Faster)]
px.density_heatmap(FasterV, x=FasterV['X'], y=FasterV['Y'])

In [ ]:
counts, bins = np.histogram(FasterV.Timestamp, bins=range(FasterV["Timestamp"].min(), FasterV["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

### Slow-moving tourists (high gap between actions)

In [ ]:
inds3 =  np.argsort(med_td)
Slower = np.array(visitors_td)[inds3[-150:]]
SlowerV = Sunday_II.loc[Sunday_II['id'].isin(Slower)]
px.density_heatmap(SlowerV, x=SlowerV['X'], y=SlowerV['Y'])

In [ ]:
counts, bins = np.histogram(SlowerV.Timestamp, bins=range(SlowerV["Timestamp"].min(), SlowerV["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})

In [ ]:
ShowTimeS = time.mktime(time.strptime("2014-6-08 09:00:00", "%Y-%m-%d %H:%M:%S"))
ShowTimeE = time.mktime(time.strptime("2014-6-08 12:00:00", "%Y-%m-%d %H:%M:%S"))

CheckPointM = Sunday_II.copy()
SuspiciousIN = CheckPointM[   (CheckPointM["Timestamp"]>=ShowTimeS) & (CheckPointM["Timestamp"]<=ShowTimeE)                                                       ]
SuspiciousIN = SuspiciousIN[  (SuspiciousIN["X"]>=20) &  (SuspiciousIN["Y"]>=30) ]
SuspiciousIN = SuspiciousIN[  (SuspiciousIN["X"]<=40) &  (SuspiciousIN["Y"]<=40) ]
len(SuspiciousIN.id.unique())

In [ ]:
ShowTimeS2 = time.mktime(time.strptime("2014-6-08 14:00:00", "%Y-%m-%d %H:%M:%S"))
ShowTimeE2 = time.mktime(time.strptime("2014-6-08 17:00:00", "%Y-%m-%d %H:%M:%S"))

CheckPointA = Sunday_II.copy()
SuspiciousIN2 = CheckPointA[   (CheckPointA["Timestamp"]>=ShowTimeS2) & (CheckPointA["Timestamp"]<=ShowTimeE2)                                                       ]
SuspiciousIN2 = SuspiciousIN2[  (SuspiciousIN2["X"]>=20) &  (SuspiciousIN2["Y"]>=30) ]
SuspiciousIN2 = SuspiciousIN2[  (SuspiciousIN2["X"]<=40) &  (SuspiciousIN2["Y"]<=40) ]
len(SuspiciousIN2.id.unique())

In [ ]:
SuspiciousGroup =  list(set( list(SuspiciousIN.id.unique()) ) & set(list(SuspiciousIN2.id.unique()))) 
GSP = Sunday_II.loc[Sunday_II['id'].isin(SuspiciousGroup)]
len(SuspiciousGroup)

In [ ]:
import csv
with open("SundayAreaCheck","w") as f:
    write = csv.writer(f)
    write.writerow(SuspiciousGroup)

In [ ]:
px.density_heatmap(GSP, x='X', y='Y')

In [ ]:
counts, bins = np.histogram(GSP.Timestamp, bins=range(GSP["Timestamp"].min(), GSP["Timestamp"].max(), 60))
bins = bins[1:]
px.line(['timestamp', 'count'], x=bins, y=counts, labels={'x':'timestamp', 'y':'count'})